In [1]:
# %run /Users/denis/Documents/Projects/scripts/init.ipy
%run /Users/maayanlab/Documents/init.ipy
os.chdir('../../')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Infiles

In [2]:
infile = 's1-data.dir/HMS_Datasets-merged_filtered.txt'

### Computation

In [3]:
# Get expression data
expression_dataframe = pd.read_table(infile)

# Genes
genes = [x for x in expression_dataframe.columns if x not in ['DrugName', 'Conc', 'well', 'fraction', 'timepoint'] and 'DAPI' not in x and 'CellMask' not in x and 'MitoTracker' not in x]

In [ ]:
# Loop through genes
for i, gene_symbol in enumerate(genes):
    
    # Print
    print 'Doing '+gene_symbol+' ('+str(i+1)+'/'+str(len(genes))+' genes)...'

    # Get expression data
    gene_expression_dataframe = expression_dataframe[['DrugName', 'Conc', 'fraction', 'timepoint', gene_symbol]]
    gene_expression_dataframe[gene_symbol] = np.log10(expression_dataframe[gene_symbol]+1)

    # Get conditions
    condition_dataframe = gene_expression_dataframe.drop(gene_symbol, axis=1).drop_duplicates().query('DrugName!="DMSO"').reset_index(drop=True)

    # Set index
    gene_expression_dataframe.set_index(['DrugName', 'Conc', 'fraction', 'timepoint'], inplace=True)

    # Seaborn settings
    sns.set_context('poster')
    sns.set_style('whitegrid')

    # Loop through conditions
    for drug_name, concentration, fraction, timepoint in condition_dataframe.as_matrix():

        # Get values to compare
        treated = [{'Expression (log10)': x, 'Treatment': drug_name} for x in gene_expression_dataframe.loc[drug_name, concentration, fraction, timepoint][gene_symbol]]
        untreated = [{'Expression (log10)': x, 'Treatment': 'DMSO'} for x in gene_expression_dataframe.loc['DMSO', 0, fraction, timepoint][gene_symbol]]

        # Get dataframe
        plot_dataframe = pd.DataFrame(treated+untreated)

        # Plot
        fig, ax = plt.subplots(figsize=(8.5,6.5))
        concentration = str(int(concentration)) if concentration in [1, 10] else concentration
        sns_plot = sns.violinplot(data=plot_dataframe, x="Treatment", y="Expression (log10)").get_figure()
        plt.title('{gene_symbol} | {fraction} | {concentration}uM, {timepoint}'.format(**locals()))
        sns_plot.subplots_adjust(top=0.93)
        sns_plot.savefig("../mcf10a-cycif-browser/images/differential_expression/{gene_symbol}-{drug_name}-{concentration}-{fraction}-{timepoint}.png".format(**locals()))
        sns_plot.clf()
        plt.close()

Doing ERK-1 (1/19 genes)...


/Users/maayanlab/Documents/Computation/my-venv/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/maayanlab/Documents/Computation/my-venv/lib/python2.7/site-packages/ipykernel_launcher.py:25: PerformanceWarning:

indexing past lexsort depth may impact performance.

/Users/maayanlab/Documents/Computation/my-venv/lib/python2.7/site-packages/ipykernel_launcher.py:26: PerformanceWarning:

indexing past lexsort depth may impact performance.



Doing Cyclin-D1 (2/19 genes)...
Doing FOXO3a (3/19 genes)...
Doing PCNA (4/19 genes)...
Doing KI-67 (5/19 genes)...
Doing EGFR (6/19 genes)...
